In [1]:
!pip install ipywebrtc
!pip install datasets
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
Paths used for configuration of notebook: 
    	/root/.jupyter/nbconfig/notebook.json
Paths used for configuration of notebook: 
    	
      - Validating: OK
Paths used for configuration of notebook: 
    	/root/.jupyter/nbconfig/notebook.json


In [2]:
import pickle
import numpy as np
from transformers import AutoProcessor, AutoModelForCTC
from ipywebrtc import AudioRecorder, CameraStream
from IPython.display import Audio
import torchaudio
import torch
import librosa
from google.colab import output
output.enable_custom_widget_manager()

In [3]:
model_name = 'ctaguchi/wav2vec2-large-xlsr-japlmthufielta-ipa-plus-2000'
processor = AutoProcessor.from_pretrained(model_name)
model = AutoModelForCTC.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of the model checkpoint at ctaguchi/wav2vec2-large-xlsr-japlmthufielta-ipa-plus-2000 were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- T

In [4]:
def load_audio(recorder):
  with open('recording.webm', 'wb') as f:
      f.write(recorder.audio.value)
  !ffmpeg -i recording.webm -ac 1 -f wav file.wav -y -hide_banner -loglevel panic
  return torchaudio.load("file.wav")

In [5]:
camera = CameraStream(constraints={'audio': True,'video': False})
recorder = AudioRecorder(stream=camera)
recorder

AudioRecorder(audio=Audio(value=b'', format='webm'), stream=CameraStream(constraints={'audio': True, 'video': …

In [8]:
sig, sr = load_audio(recorder)

# Resample audio to the model's training sampling rate (16000)
sig_resampled = librosa.resample(np.array(sig), orig_sr=sr, target_sr=16000)

# Update the sampling rate variable
sr = 16000

# Process the resampled audio
inputs = processor(sig_resampled, sampling_rate=sr, return_tensors="pt")

with torch.no_grad():
  logits = model(**inputs).logits

predicted_ids = torch.argmax(logits, dim=-1)

# transcribe speech
transcription = processor.batch_decode(predicted_ids)
transcription[0]

'hɛlɔ wɔɻld'